In [4]:
import json
import jsonlines

# 1. 파일 경로 정의
locations_file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_extracted_locations_result_ten_year_all.jsonl'
preprocessing_file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/re_final_preprocessing.json'
output_file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_final_combined_data_ten_year.json'

# 2. locations 파일에서 데이터 매핑
locations_map = {}
with jsonlines.open(locations_file_path) as reader:
    for obj in reader:
        locations_map[obj['id_']] = obj['locations']

# 3. preprocessing 파일 불러오기 (리스트 형태라고 가정)
with open(preprocessing_file_path, 'r', encoding='utf-8') as f:
    preprocessing_list = json.load(f)

# 4. id_를 기준으로 preprocessing 데이터를 빠르게 찾기 위한 딕셔너리 생성
# 이 단계는 preprocessing_list가 매우 클 때 효율성을 높여줍니다.
preprocessing_map = {doc.get('id_'): doc for doc in preprocessing_list}

# 5. locations_map의 id_를 기준으로 데이터 결합
combined_data_list = []
for doc_id, locations in locations_map.items():
    # preprocessing_map에서 해당 id_를 가진 문서를 찾습니다.
    preprocessing_doc = preprocessing_map.get(doc_id)
    
    # 해당 문서가 존재하는 경우에만 결합
    if preprocessing_doc:
        pub_date = preprocessing_doc.get('metadata', {}).get('pubDate')
        title = preprocessing_doc.get('metadata', {}).get('title')
        url = preprocessing_doc.get('metadata', {}).get('url')
        
        new_data = {
            'id_': doc_id,
            'title': title,
            'url': url,
            'pubDate': pub_date,
            'locations': locations
        }
        combined_data_list.append(new_data)

# 6. 새로운 JSON 파일로 저장
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(combined_data_list, f, indent=4, ensure_ascii=False)

print(f"새로운 JSON 파일이 '{output_file_path}'에 성공적으로 저장되었습니다.")

새로운 JSON 파일이 '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_final_combined_data_ten_year.json'에 성공적으로 저장되었습니다.


In [5]:
import json
import os
from datetime import datetime

# 1. 파일 경로 정의
input_file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_final_combined_data_ten_year.json'
output_dir = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/'

# 2. 출력 디렉터리 생성 (존재하지 않을 경우)
os.makedirs(output_dir, exist_ok=True)
print(f"출력 디렉터리 '{output_dir}'를 확인하거나 생성했습니다.")

# 3. 전체 데이터 불러오기
try:
    with open(input_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
except FileNotFoundError:
    print(f"오류: 입력 파일이 존재하지 않습니다. 경로를 확인해주세요: {input_file_path}")
    exit()

# 4. pubDate 기준으로 데이터 정렬
sorted_data = sorted(data, key=lambda x: datetime.strptime(x['pubDate'], '%Y-%m-%d %H:%M:%S'))

# 5. 연도별로 데이터 그룹화
grouped_by_year = {}
for item in sorted_data:
    try:
        year = datetime.strptime(item['pubDate'], '%Y-%m-%d %H:%M:%S').year
        if year not in grouped_by_year:
            grouped_by_year[year] = []
        grouped_by_year[year].append(item)
    except (KeyError, ValueError):
        print(f"경고: 유효하지 않은 'pubDate' 형식을 가진 항목이 발견되어 건너뜁니다: {item}")
        continue

# 6. 각 연도별 데이터를 지정된 폴더에 JSON 파일로 저장
for year, items in grouped_by_year.items():
    output_file_path = os.path.join(output_dir, f'new_combined_data_{year}.json')
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(items, f, indent=4, ensure_ascii=False)
    print(f"'{output_file_path}' 파일에 {year}년 데이터 {len(items)}개가 저장되었습니다.")

print("\n모든 파일 분할 및 저장이 완료되었습니다. ✅")

출력 디렉터리 '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/'를 확인하거나 생성했습니다.
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/new_combined_data_2016.json' 파일에 2016년 데이터 7385개가 저장되었습니다.
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/new_combined_data_2017.json' 파일에 2017년 데이터 7924개가 저장되었습니다.
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/new_combined_data_2018.json' 파일에 2018년 데이터 7224개가 저장되었습니다.
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/new_combined_data_2019.json' 파일에 2019년 데이터 5786개가 저장되었습니다.
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/new_combined_data_2020.json' 파일에 2020년 데이터 3202개가 저장되었습니다.
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/new_combined_data_2021.json' 파일에 2021년 데이터 2218개가 저장되었습니다.
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year/new_combined_data_2022.json' 